Sentiment Analysis
- to extract subjective information and rate the sentiment according to review Text and compare to orignoal overall score to find out accuracy



In [1]:
import os, json, gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import string
import nltk

from nltk.corpus import wordnet, stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/li.wany/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/li.wany/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/li.wany/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/li.wany/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/li.wany/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /home/li.wany/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
df = pd.read_csv('reasonable_reviews.csv')
# Check the population size (number of rows) in your DataFrame
print("Population size:", len(df))


Population size: 38859


/shared/centos7/anaconda3/2020.02/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We only forcus on overall and reviewText column, and we can see there is imbalanced data from overall 1 to overall 5. So we need to sort out an equal number of obervation for each class. 

In [4]:
# drop any rows w/ missing values
df = df.dropna(subset=["reviewText"])
print("Population size:", len(df))
#discover the actual counts
print(df.overall.value_counts())

print("Population size:", len(df))

# set sample size to labels w/ minimum count
sample_size = 500
# Collect samples for each class in a list
samples_list = [df[df.overall == label].sample(n=sample_size, random_state=1) for label in df.overall.unique()]

# Concatenate all the samples into one DataFrame
df_equal_overall = pd.concat(samples_list, ignore_index=True)

Population size: 38859
5.0    31311
4.0     4643
3.0     1860
2.0      540
1.0      505
Name: overall, dtype: int64
Population size: 38859


Fundamental Preprocess for NLP : lowercasing, punctuations removal, and removal of stopwords. 

In [5]:
stopwords_list = stopwords.words('english')

def ReviewProcessing(df):
  # remove non alphanumeric
  df['review_cleaned'] = df.reviewText.str.replace('[^a-zA-Z0-9 ]', '')
  # lowercase
  df.review_cleaned = df.review_cleaned.str.lower()
  # split into list
  df.review_cleaned = df.review_cleaned.str.split(' ')
  # remove stopwords
  df.review_cleaned = df.review_cleaned.apply(lambda x: [item for item in x if item not in stopwords_list])
  return df

Lemmatiazation for NLP: the process of grouping inflected words into a root word. 

In [6]:
def get_wordnet_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

  return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = nltk.stem.WordNetLemmatizer()
def get_lemmatize(sent):
  return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])


In [7]:
clean_data = ReviewProcessing(df_equal_overall)
clean_data.review_cleaned = clean_data.review_cleaned.apply(' '.join)
clean_data['review_cleaned_lemmatized'] = clean_data.review_cleaned.apply(get_lemmatize)

Implementing TF-IDF Weighting : to measure how important or relevant a term is within the sentence

In [8]:
from sklearn.naive_bayes import MultinomialNB

nb = Pipeline([('vectorize', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

Stochastic Gradient Descent: SGD algorithm computes the minimum of the coust function 

In [9]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier()),
               ])

Logistic Regression Classifier: categorizing dazta into binary or multiplr groups with discrete prediction value. 

In [10]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=500)),
               ])

Split dataset into train and test sets

In [11]:
x = clean_data['review_cleaned_lemmatized']
y = clean_data['overall']
X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2, stratify=y, random_state = 44)


Return the accuracy socre, confusion matrix and classification repost for each report

In [12]:
# Naive Bayes
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print(accuracy_score(y_test, y_pred_nb))
print(confusion_matrix(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# SGD Classifier
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
print(accuracy_score(y_test, y_pred_sgd))
print(confusion_matrix(y_test, y_pred_sgd))
print(classification_report(y_test, y_pred_sgd))

# Logistic Regression
logreg.fit(X_train, y_train)
y_pred_log = logreg.predict(X_test)
print(accuracy_score(y_test, y_pred_log))
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

0.438
[[44 25 18 10  3]
 [ 9 36 32 10 13]
 [ 5 17 49 24  5]
 [ 2  7 26 39 26]
 [ 2  2  8 37 51]]
              precision    recall  f1-score   support

         1.0       0.71      0.44      0.54       100
         2.0       0.41      0.36      0.39       100
         3.0       0.37      0.49      0.42       100
         4.0       0.33      0.39      0.35       100
         5.0       0.52      0.51      0.52       100

    accuracy                           0.44       500
   macro avg       0.47      0.44      0.44       500
weighted avg       0.47      0.44      0.44       500

0.402
[[45 21 16 10  8]
 [28 29 20 12 11]
 [ 9 17 36 28 10]
 [ 5  6 21 31 37]
 [ 3  1  7 29 60]]
              precision    recall  f1-score   support

         1.0       0.50      0.45      0.47       100
         2.0       0.39      0.29      0.33       100
         3.0       0.36      0.36      0.36       100
         4.0       0.28      0.31      0.30       100
         5.0       0.48      0.60      0.53   

In [13]:

from sklearn.model_selection import GridSearchCV

grid=[{'clf__solver': ['lbfgs', 'sag', 'saga'],
       'clf__C': [0.01, 0.1, 1]}]
lr = GridSearchCV(logreg, param_grid = grid, cv = 5, scoring='accuracy', verbose = 1, n_jobs = -1)
best_model = lr.fit(X_train, y_train)

print(best_model.best_estimator_)
print(best_model.best_score_)

y_pred_grid = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid))
print(accuracy_score(y_test, y_pred_grid))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  45 | elapsed:    3.2s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    5.3s finished


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=0.1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
  